In [ ]:
from qdrant_client import QdrantClient, models

In [ ]:
client = QdrantClient(":memory:")

client.create_collection(
    "test",
    vectors_config={},
    sparse_vectors_config={
        'test-sparse': models.SparseVectorParams(
            index=models.SparseIndexParams(
                on_disk=False
            )
        )
    }
)

True

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

docs = [
    "This is a document about science: physics, chemistry, biology",
    "This is a document about arts: painting, sculpture, music",
    "This is a document about programming: python, c++, java",
]

vectorizer = TfidfVectorizer()
vectorizer.fit(docs)

vectorizer_dense = lambda x: list(vectorizer.transform([x]).toarray().flatten())

def vectorizer_dense(text):
    svector = vectorizer.transform([text])
    return list(svector.indices), list(svector.data)

In [7]:
vectorizer_dense('physics')

([9], [1.0])

In [8]:
from langchain_community.retrievers import QdrantSparseVectorRetriever

retriever = QdrantSparseVectorRetriever(
    client=client,
    collection_name="test",
    sparse_vector_name="test-sparse",
    sparse_encoder=vectorizer_dense
)

/tmp/ipykernel_58482/324443383.py:3: LangChainDeprecationWarning: The class `QdrantSparseVectorRetriever` was deprecated in LangChain 0.2.16 and will be removed in 0.5.0. Use Qdrant vector store now supports sparse retrievals natively. Use langchain_qdrant.QdrantVectorStore#as_retriever() instead. Reference: https://python.langchain.com/v0.2/docs/integrations/vectorstores/qdrant/#sparse-vector-search instead.
  retriever = QdrantSparseVectorRetriever(


In [9]:
retriever.add_texts(
    texts=docs,
    # ids=range(len(docs))
)

['4ca37085d5174ee391453724c3d9207c',
 '69fe84271caa4903ad785f6a04878196',
 '80a205fe9bca4a268dd1285bc4ce1402']

In [10]:
retriever.invoke("physics")

[Document(metadata={'_id': '4ca37085d5174ee391453724c3d9207c', '_collection_name': 'test'}, page_content='This is a document about science: physics, chemistry, biology')]

In [11]:
retriever.add_texts(texts=["Black holes are a mystery, they are a fascinating field of physics."])

['950d0ecc21b841d48aebf99d065427bf']

In [12]:
retriever.invoke("physics")

[Document(metadata={'_id': '950d0ecc21b841d48aebf99d065427bf', '_collection_name': 'test'}, page_content='Black holes are a mystery, they are a fascinating field of physics.'),
 Document(metadata={'_id': '4ca37085d5174ee391453724c3d9207c', '_collection_name': 'test'}, page_content='This is a document about science: physics, chemistry, biology')]